In [1]:
import os
import shutil
import random

SOURCE = r"C:\temp"
DEST = r"jetbot_groundview_dataset"
TRAIN_FRAC = 0.8  # 80% train, 20% val

img_dir = os.path.join(DEST, 'images')
lbl_dir = os.path.join(DEST, 'labels')

for split in ['train', 'val']:
    os.makedirs(os.path.join(img_dir, split), exist_ok=True)
    os.makedirs(os.path.join(lbl_dir, split), exist_ok=True)

# Gather all valid images (ignore debug, require matching txt)
all_images = []
for fname in os.listdir(SOURCE):
    if fname.endswith('.png') and 'debug' not in fname:
        base = fname[:-4]
        txt = base + '.txt'
        if os.path.exists(os.path.join(SOURCE, txt)):
            all_images.append(base)

random.shuffle(all_images)
split_idx = int(TRAIN_FRAC * len(all_images))
train_bases = all_images[:split_idx]
val_bases = all_images[split_idx:]

def copy_split(split_bases, split):
    for base in split_bases:
        img_src = os.path.join(SOURCE, base + '.png')
        lbl_src = os.path.join(SOURCE, base + '.txt')
        img_dst = os.path.join(img_dir, split, base + '.png')
        lbl_dst = os.path.join(lbl_dir, split, base + '.txt')
        shutil.copy2(img_src, img_dst)
        shutil.copy2(lbl_src, lbl_dst)

copy_split(train_bases, 'train')
copy_split(val_bases, 'val')

print(f"Dataset prepared in {DEST}. Train: {len(train_bases)} | Val: {len(val_bases)}")


Dataset prepared in jetbot_groundview_dataset. Train: 24000 | Val: 6000


In [ ]:
import os
import random
import numpy as np
import torch
from ultralytics import YOLO

# ========== SETTINGS ==========
DATA_YAML = "jetbot_groundview_dataset/data.yaml"      # Path to your data.yaml
MODEL = "yolov8n.pt"                      # Or 'yolov8s.pt' for larger model
EPOCHS = 100
IMAGE_SIZE = 512
BATCH = 16                                 # Lower if OOM, higher if possible
DEVICE = 0                                # 'cpu' or 0 for first GPU
PROJECT = "runs/train"
NAME = "jetbot_groundview"
SEED = 42

# ========== SEED FOR REPRODUCIBILITY ==========
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ========== TRAINING ==========
model = YOLO(MODEL)

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH,
    device=DEVICE,
    project=PROJECT,
    name=NAME,
    seed=SEED,
    workers=4,
    optimizer='SGD',        # Default (recommended)
    pretrained=True,
    val=True,
    save=True,
    plots=True,
    exist_ok=True,
    verbose=True
)

print("\n=== TRAINING FINISHED ===")
print("Results directory:", results.save_dir)


New https://pypi.org/project/ultralytics/8.3.145 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=jetbot_groundview_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=jetbot_groundview, nbs=64, nms=Fa

train: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\labels.cache... 21000 images, 0 backgrounds, 10957 corrupt: 100%|██████████| 21000/21000 [00:00<?, ?it/s]

train: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\images\screenshot_00000.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2901      1.5408]
train: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\images\screenshot_00015.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.4298]
train: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\images\screenshot_00040.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1301]
train: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\images\screenshot_00097.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.7637      1.5819]
train: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\train\images\screen

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 332.556.1 MB/s, size: 177.7 KB)


val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\labels\eval_debug.cache... 6000 images, 100 backgrounds, 3157 corrupt: 100%|██████████| 6100/6100 [00:00<?, ?it/s]

val: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\screenshot_00078.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.4259      1.0841]
val: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\screenshot_00092.png: ignoring corrupt image/label: negative label values [   -0.28856]
val: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\screenshot_00125.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2853]
val: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\screenshot_00205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3778]
val: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\screenshot_00272.png: ignoring corrupt image/label: non-norm

Plotting labels to runs\train\jetbot_groundview\labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to runs\train\jetbot_groundview
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      3.04G      1.823      2.968      1.927         72        512: 100%|██████████| 314/314 [01:53<00:00,  2.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:32<00:00,  1.40it/s]


                   all       2943       3583      0.705      0.317      0.252      0.156

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100      3.46G      1.545      2.429      1.649         73        512:  50%|█████     | 158/314 [00:23<00:19,  7.92it/s]

In [ ]:
import os
from ultralytics import YOLO

# Path to your trained model
MODEL_PATH = "runs/train/jetbot_groundview/weights/best.pt"   # Update path if needed!
TRAIN_IMG_DIR = "jetbot_groundview_dataset/images/train"

# Load model
model = YOLO(MODEL_PATH)

# List all PNG images (ignoring 'debug')
image_files = [f for f in os.listdir(TRAIN_IMG_DIR) if f.endswith('.png') and 'debug' not in f]

print(f"Testing {len(image_files)} training images...")

for i, fname in enumerate(image_files):
    img_path = os.path.join(TRAIN_IMG_DIR, fname)
    results = model(img_path)
    
    print(f"\nImage: {fname}")
    for box, cls, conf in zip(results[0].boxes.xyxy, results[0].boxes.cls, results[0].boxes.conf):
        print(f"  - Class: {int(cls)}, Confidence: {conf:.2f}, Box: {box.cpu().numpy()}")
    
    # Show prediction
    results[0].show()   # Opens a window for each image (close to continue)

    # Optionally, to save annotated image to disk:
    results[0].save(filename=f"inference_train_{i}.png")

print("\nDone! Inspect visually and verify nearly perfect predictions on all training images.")
